In [2]:
import pandas as pd 
import torch
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
import torch
from torch import nn
from sklearn.metrics import classification_report


ImportError: cannot import name 'maybe_sync' from 'fsspec.asyn' (c:\Users\Wen\anaconda3\Lib\site-packages\fsspec\asyn.py)

## 1. Data prepration 

In [38]:
model_name = "distilbert-base-uncased"


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [39]:
data_path = "C:/Users/Wen/OneDrive/Documents/GitHub/Portfolio_Project/Deep Learning/Shinobi Naruto Insights -NLP/Web Scraping/jutsu.jsonl"

df = pd.read_json(data_path, lines=True)
df.head()

,jutsu_name,jutsu_type,jutsu_description
0,10 Hit Combo,Taijutsu,Lars punches the opponent before striking them...
1,Avalanche Dance,"Taijutsu, Shurikenjutsu","Haku delivers a string of kicks, punches, and ..."
2,Attack of the Twin Demons,"Kekkei Genkai, Ninjutsu",This technique allows Ukon to inhabit his brot...
3,Bando's Chakra Armour,Ninjutsu,Bando activates his chakra to surround him in ...
4,Ball of Light Technique,Ninjutsu,The user gathers a ball of concentrated light ...


In [40]:
def jutsu_types(jutsu):
    if "Taijutsu" in jutsu:
        return "Taijutsu"
    if "Ninjutsu" in jutsu:
        return "Ninjutsu"
    if "Genjutsu" in jutsu:
        return "Genjutsu"
    
    return None

df["3_types_of_jutsu"] = df["jutsu_type"].apply(jutsu_types)
df["3_types_of_jutsu"].value_counts()

3_types_of_jutsu
Ninjutsu    2009
Taijutsu     628
Genjutsu      79
Name: count, dtype: int64

In [41]:
df.columns

Index(['jutsu_name', 'jutsu_type', 'jutsu_description', '3_types_of_jutsu'], dtype='object')

In [42]:
df['text'] = df['jutsu_name']+ '. '+df['jutsu_description']
df['jutsu'] = df['3_types_of_jutsu']
df= df[['text','jutsu']].dropna()
print(df)

                                                   text     jutsu
0     10 Hit Combo. Lars punches the opponent before...  Taijutsu
1     Avalanche Dance. Haku delivers a string of kic...  Taijutsu
2     Attack of the Twin Demons. This technique allo...  Ninjutsu
3     Bando's Chakra Armour. Bando activates his cha...  Ninjutsu
4     Ball of Light Technique. The user gathers a ba...  Ninjutsu
...                                                 ...       ...
2867  Absolute: Fang Passing Fang. Kiba and Akamaru ...  Taijutsu
2868  16 Hit Combo. A very effective move, Ino uses ...  Taijutsu
2869  1000 Metre Punch. The user focuses a large amo...  Taijutsu
2870  100% Single Punch. Tsunade gathers large amoun...  Taijutsu
2871  100 Metre Punch. A shorter version of the 1000...  Taijutsu

[2716 rows x 2 columns]


### 1.1 Clean Data 

In [59]:
def put_line_breaks(self, text):
    text = text.replace('\n', ' ')
    return text

In [60]:
class Cleaner():
    def __init__(self):
        pass
    
    def remove_html_tags(self, text):
        """Remove HTML tags using BeautifulSoup."""
        cleantext = BeautifulSoup(text, "lxml").text
        return cleantext
    
    def clean(self, text):
        """Clean text by removing HTML tags and standardizing white space."""
        text = self.remove_html_tags(text)
        text = ' '.join(text.split())
        return text


In [73]:
cleaned_data = Cleaner()
df['text_cleaned'] = df["text"].apply(cleaned_data.clean)
print(df['text_cleaned'])

C:\Users\Wen\AppData\Local\Temp\ipykernel_5512\3021304313.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleantext = BeautifulSoup(text, "lxml").text


0       10 Hit Combo. Lars punches the opponent before...
1       Avalanche Dance. Haku delivers a string of kic...
2       Attack of the Twin Demons. This technique allo...
3       Bando's Chakra Armour. Bando activates his cha...
4       Ball of Light Technique. The user gathers a ba...
                              ...                        
2867    Absolute: Fang Passing Fang. Kiba and Akamaru ...
2868    16 Hit Combo. A very effective move, Ino uses ...
2869    1000 Metre Punch. The user focuses a large amo...
2870    100% Single Punch. Tsunade gathers large amoun...
2871    100 Metre Punch. A shorter version of the 1000...
Name: text_cleaned, Length: 2716, dtype: object


### 1.2 Label Encoder

In [74]:
encoder = LabelEncoder()
encoder.fit(df["jutsu"].tolist())
df['label'] = encoder.transform(df["jutsu"].tolist())
df.head()


,text,jutsu,text_cleaned,label
0,10 Hit Combo. Lars punches the opponent before...,Taijutsu,10 Hit Combo. Lars punches the opponent before...,2
1,Avalanche Dance. Haku delivers a string of kic...,Taijutsu,Avalanche Dance. Haku delivers a string of kic...,2
2,Attack of the Twin Demons. This technique allo...,Ninjutsu,Attack of the Twin Demons. This technique allo...,1
3,Bando's Chakra Armour. Bando activates his cha...,Ninjutsu,Bando's Chakra Armour. Bando activates his cha...,1
4,Ball of Light Technique. The user gathers a ba...,Ninjutsu,Ball of Light Technique. The user gathers a ba...,1


### 1.3 Class Weights

In [65]:
class_weights = compute_class_weight('balanced',
                     classes=sorted(df['label'].unique().tolist()),
                     y=df['label'].tolist()).tolist()

print(class_weights)

[11.459915611814345, 0.45063879210220675, 1.4416135881104033]


### 1.4 Train Testing Split

In [67]:
df_train,df_test = train_test_split(df,test_size=0.2,stratify=df['label'])


### 1.5 Convert to HuggingFace Datast

In [68]:
train_data = Dataset.from_pandas(df_train)
test_data = Dataset.from_pandas(df_test)

NameError: name 'Dataset' is not defined

### 1.6 Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text_cleaned"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)



## 2. Initialize Model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

## 3. Train Model 

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).to(device=device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy = "epoch",
    logging_strategy="epoch"
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
trainer.save_model('jutsu_model')

## 4. Evaluate Model 

In [ ]:
preds = trainer.predict(tokenized_train)
preds = np.argmax(preds[:3][0],axis=1)
actual = df_train['label'].tolist()
print(classification_report(actual,preds))

In [ ]:
preds = trainer.predict(tokenized_test)
preds = np.argmax(preds[:3][0],axis=1)
actual = df_test['label'].tolist()
print(classification_report(actual,preds))